In [1]:
import numpy as np
import pandas as pd

In [2]:
from lightautoml_gpu.reader.gpu.cudf_reader import CudfReader
from lightautoml_gpu.reader.base import PandasToPandasReader

from lightautoml_gpu.transformers.base import SequentialTransformer

from lightautoml_gpu.pipelines.utils import get_columns_by_role

from lightautoml_gpu.transformers.gpu import numeric_gpu, categorical_gpu, datetime_gpu
from lightautoml_gpu.transformers import numeric, categorical, datetime

from lightautoml_gpu.tasks import Task
from lightautoml_gpu.automl.presets.tabular_presets import TabularAutoML, TabularUtilizedAutoML

import pandas as pd
import time
from sklearn.model_selection import train_test_split

from dask.distributed import Client
from dask_cuda import LocalCUDACluster
import cudf

In [3]:
features = pd.read_csv('../../data/small_new/multilabel/train Data.csv')
labels = pd.read_csv('../../data/small_new/multilabel/train labels.csv')
labels.drop(columns='id', inplace=True)
data = pd.concat([features, labels], axis=1)
data = data[['n_0000','n_0001','n_0002','n_0003',
             'n_0004','n_0005','n_0006','n_0007',
             'service_a', 'service_b', 'service_c', 
             'service_d', 'service_e', 'service_f',
             'service_g', 'service_h', 'service_i',
             'service_j', 'service_k', 'service_l',
             'service_m', 'service_n']]

tr_data, te_data = train_test_split(
    data, 
    test_size=0.2,  
    random_state=42
)

roles = {
    "target": {'service_a', 'service_b', 'service_c',
               'service_d', 'service_e', 'service_f',
               'service_g', 'service_h', 'service_i',
               'service_j', 'service_k', 'service_l',
               'service_m', 'service_n'},
    "drop" : ['id']
}

/tmp/ipykernel_5476/705777730.py:1: DtypeWarning: Columns (329,331,333,336,338,344,345,346,348,354,355,356,357,358,361,362,364,367,372,377,380,383,385,387,390,392,399,400,406,408,409,413,416,418,419,431,433,437,438,442,448,449,450,453,457,464,473,478,479,481,483,485,486,489,492,495,496,497,498,499,500,503,507,508,510,511,514,515,517,519,520,521,522,523,524,526,527,530,534,537,538,539,541,544,547,548,549,550,551,557,558,560,564,569,572,573,576,577,579,583,596,597,598,600,601,602,604,605,606,608,609,613,619,620,625,627,628,629,631,632,633,636,641,642,643,645,646,647,648,651,655,661,662,665,668,675,676,679,682,685,687,689,690,691,694,698,701,702,703,706,711,712,713,719,720,721,733,735,737,738,742,746,747,748,749,752,754,755,760,764,768,770,781,782,789,797,807,812,814,817,818,822,823,824,825,832,840,843,844,845,850,853,857,858,861,867,868,873,874,876,877,879,880,881,883,886,890,893,897,899,900,901,902,904,905,908,909,910,912,913,914,915,916,922,923,931,933,935,937,939,942,943,946,951,955,9

In [4]:
adv_roles = True

## Imports (for potential use)

In [5]:
# Imports from our package
from lightautoml_gpu.automl.base import AutoML

from lightautoml_gpu.automl.presets.gpu.tabular_gpu_presets import TabularAutoMLGPU, TabularUtilizedAutoMLGPU
from lightautoml_gpu.tasks import Task

from lightautoml_gpu.pipelines.features.gpu.lgb_pipeline_gpu import LGBSimpleFeaturesGPU, LGBAdvancedPipelineGPU
from lightautoml_gpu.pipelines.features.gpu.linear_pipeline_gpu import LinearFeaturesGPU

from lightautoml_gpu.pipelines.features.lgb_pipeline import LGBSimpleFeatures, LGBAdvancedPipeline
from lightautoml_gpu.pipelines.features.linear_pipeline import LinearFeatures


from lightautoml_gpu.ml_algo.gpu.boost_cb_gpu import BoostCBGPU
from lightautoml_gpu.ml_algo.gpu.boost_xgb_gpu import BoostXGB
from lightautoml_gpu.ml_algo.gpu.linear_gpu import LinearLBFGSGPU

from lightautoml_gpu.ml_algo.boost_cb import BoostCB
from lightautoml_gpu.ml_algo.linear_sklearn import LinearLBFGS


from lightautoml_gpu.pipelines.ml.base import MLPipeline
from lightautoml_gpu.pipelines.selection.importance_based import ModelBasedImportanceEstimator, ImportanceCutoffSelector

## TabularAutoML

In [6]:
task = Task('multilabel')

multilabel isn`t supported in lgb


In [7]:
automl = TabularAutoML(
    task = task, 
    timeout = 3600,
    cpu_limit = 4,
    reader_params = {'n_jobs': 4, 'cv': 3, 'random_state': 42},
    general_params = {'use_algos': [['linear_l2', 'cb']]}
)

In [8]:
oof_pred = automl.fit_predict(data, roles = roles, verbose = 2)

[22:14:24] Stdout logging level is INFO2.
[22:14:24] Copying TaskTimer may affect the parent PipelineTimer, so copy will create new unlimited TaskTimer
[22:14:24] Task: multilabel

[22:14:24] Start automl preset with listed constraints:
[22:14:24] - time: 3600.00 seconds
[22:14:24] - CPU: 4 cores
[22:14:24] - memory: 16 GB

[22:14:24] Train data shape: (14644, 22)



/home/rishat/miniconda3/envs/rapids-22.10/lib/python3.9/site-packages/lightautoml_gpu/reader/guess_roles.py:56: RuntimeWarning: invalid value encountered in float_scalars
  gini_sum = a_c.sum() / a_s.sum() - (n + 1) / 2.0
/home/rishat/miniconda3/envs/rapids-22.10/lib/python3.9/site-packages/lightautoml_gpu/reader/guess_roles.py:56: RuntimeWarning: invalid value encountered in float_scalars
  gini_sum = a_c.sum() / a_s.sum() - (n + 1) / 2.0


[22:14:28] Feats was rejected during automatic roles guess: []
[22:14:28] Layer 1 train process start. Time left 3596.57 secs
[22:14:28] Start fitting Lvl_0_Pipe_0_Mod_0_LinearL2 ...
[22:14:28] ===== Start working with fold 0 for Lvl_0_Pipe_0_Mod_0_LinearL2 =====


/home/rishat/miniconda3/envs/rapids-22.10/lib/python3.9/site-packages/lightautoml_gpu/reader/guess_roles.py:56: RuntimeWarning: invalid value encountered in float_scalars
  gini_sum = a_c.sum() / a_s.sum() - (n + 1) / 2.0
/home/rishat/miniconda3/envs/rapids-22.10/lib/python3.9/site-packages/lightautoml_gpu/reader/guess_roles.py:56: RuntimeWarning: invalid value encountered in float_scalars
  gini_sum = a_c.sum() / a_s.sum() - (n + 1) / 2.0
/home/rishat/miniconda3/envs/rapids-22.10/lib/python3.9/site-packages/lightautoml_gpu/reader/guess_roles.py:56: RuntimeWarning: invalid value encountered in float_scalars
  gini_sum = a_c.sum() / a_s.sum() - (n + 1) / 2.0
/home/rishat/miniconda3/envs/rapids-22.10/lib/python3.9/site-packages/lightautoml_gpu/reader/guess_roles.py:56: RuntimeWarning: invalid value encountered in float_scalars
  gini_sum = a_c.sum() / a_s.sum() - (n + 1) / 2.0


[22:14:31] ===== Start working with fold 1 for Lvl_0_Pipe_0_Mod_0_LinearL2 =====
[22:14:34] ===== Start working with fold 2 for Lvl_0_Pipe_0_Mod_0_LinearL2 =====
[22:14:36] Fitting Lvl_0_Pipe_0_Mod_0_LinearL2 finished. score = -7.35028162615705
[22:14:36] Lvl_0_Pipe_0_Mod_0_LinearL2 fitting and predicting completed
[22:14:36] Time left 3588.06 secs



...

[22:14:52] ===== Start working with fold 1 for Lvl_0_Pipe_1_Mod_0_CatBoost =====


/home/rishat/miniconda3/envs/rapids-22.10/lib/python3.9/site-packages/catboost/core.py:1419: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  self._init_pool(data, label, cat_features, text_features, embedding_features, embedding_features_data, pairs, weight,
/home/rishat/miniconda3/envs/rapids-22.10/lib/python3.9/site-packages/catboost/core.py:1419: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  self._init_pool(data, label, cat_features, text_features, embedding_features, embedding_features_data, pairs, weight,


[22:15:00] ===== Start working with fold 2 for Lvl_0_Pipe_1_Mod_0_CatBoost =====


/home/rishat/miniconda3/envs/rapids-22.10/lib/python3.9/site-packages/catboost/core.py:1419: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  self._init_pool(data, label, cat_features, text_features, embedding_features, embedding_features_data, pairs, weight,
/home/rishat/miniconda3/envs/rapids-22.10/lib/python3.9/site-packages/catboost/core.py:1419: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  self._init_pool(data, label, cat_features, text_features, embedding_features, embedding_features_data, pairs, weight,


[22:15:08] Fitting Lvl_0_Pipe_1_Mod_0_CatBoost finished. score = -7.319639591038699
[22:15:08] Lvl_0_Pipe_1_Mod_0_CatBoost fitting and predicting completed
[22:15:08] Time left 3556.05 secs

[22:15:08] Layer 1 training completed.

[22:15:08] Blending: optimization starts with equal weights and score -7.314009267074656
[22:15:08] Blending: iteration 0: score = -7.313950612738106, weights = [0.5435342 0.4564658]
[22:15:09] Blending: iteration 1: score = -7.313950612738106, weights = [0.5435342 0.4564658]
[22:15:09] Blending: no score update. Terminated

[22:15:09] Automl preset training completed in 44.32 seconds

[22:15:09] Model description:
Final prediction for new objects (level 0) = 
	 0.54353 * (3 averaged models Lvl_0_Pipe_0_Mod_0_LinearL2) +
	 0.45647 * (3 averaged models Lvl_0_Pipe_1_Mod_0_CatBoost) 



In [9]:
task = Task('multilabel', device='gpu')

multilabel isn`t supported in lgb
[22:15:10] CatBoost uses as obj. MultiCrossEntropy.


In [10]:
automl_gpu = TabularAutoMLGPU(
    task = task, 
    timeout = 3600,
    cpu_limit = 1,
    reader_params = {'n_jobs': 1, 'cv': 3, 'random_state': 42},
    general_params = {'use_algos': [['linear_l2', 'pb']]}
)

In [11]:
oof_pred_gpu = automl_gpu.fit_predict(data, roles = roles, verbose = 2)

[22:15:11] Stdout logging level is INFO2.
[22:15:11] Task: multilabel

[22:15:11] Start automl preset with listed constraints:
[22:15:11] - time: 3600.00 seconds
[22:15:11] - CPU: 1 cores
[22:15:11] - memory: 16 GB

[22:15:11] Train data shape: (14644, 22)
[22:15:11] Feats was rejected during automatic roles guess: []
[22:15:11] Layer 1 train process start. Time left 3599.36 secs
[22:15:12] Start fitting Lvl_0_Pipe_0_Mod_0_LinearL2 ...
[22:15:12] ===== Start working with fold 0 for Lvl_0_Pipe_0_Mod_0_LinearL2 (orig) =====
[22:15:17] ===== Start working with fold 1 for Lvl_0_Pipe_0_Mod_0_LinearL2 (orig) =====
[22:15:22] ===== Start working with fold 2 for Lvl_0_Pipe_0_Mod_0_LinearL2 (orig) =====
[22:15:29] Fitting Lvl_0_Pipe_0_Mod_0_LinearL2 finished. score = -7.377731107533531
[22:15:29] Lvl_0_Pipe_0_Mod_0_LinearL2 fitting and predicting completed
[22:15:29] Time left 3581.13 secs

[22:15:52] Selector_XGB fitting and predicting completed
[22:15:52] Start fitting Lvl_0_Pipe_1_Mod_0_PB .

In [12]:
gpu_inf = automl_gpu.predict(data)

In [13]:
automl_gpu.levels[0][1].ml_algos[0].models[0].to_cpu()

In [14]:
automl_gpu.to_cpu()

multilabel isn`t supported in lgb
[22:16:01] CatBoost uses as obj. MultiCrossEntropy.
multilabel isn`t supported in lgb
[22:16:01] CatBoost uses as obj. MultiCrossEntropy.
multilabel isn`t supported in lgb
[22:16:01] CatBoost uses as obj. MultiCrossEntropy.
multilabel isn`t supported in lgb
[22:16:02] CatBoost uses as obj. MultiCrossEntropy.


  0%|          | 0/145 [00:00<?, ?it/s]

  0%|          | 0/142 [00:00<?, ?it/s]

  0%|          | 0/144 [00:00<?, ?it/s]

multilabel isn`t supported in lgb
[22:16:03] CatBoost uses as obj. MultiCrossEntropy.


In [15]:
automl_gpu.levels[0][1].ml_algos[0].__dict__

{'task': <lightautoml_gpu.tasks.base.Task at 0x7fe83e58a6a0>,
 'optimization_search_space': {},
 'freeze_defaults': True,
 'default_params': {},
 'models': [<py_boost.utils.tl_wrapper.TLPredictor at 0x7fe8465dcf40>,
 '_features': ['freq__n_0006',
  'inter__(n_0000__n_0003)',
  'inter__(n_0000__n_0004)',
  'inter__(n_0000__n_0006)',
  'inter__(n_0003__n_0004)',
  'inter__(n_0003__n_0006)',
  'inter__(n_0004__n_0006)',
  'inter__(n_0000__n_0003__n_0004)',
  'inter__(n_0000__n_0003__n_0006)',
  'inter__(n_0000__n_0004__n_0006)',
  'inter__(n_0003__n_0004__n_0006)',
  'n_0001',
  'n_0002',
  'n_0005',
  'n_0007',
  'ord__n_0000',
  'ord__n_0003',
  'ord__n_0004'],
 'timer': <lightautoml_gpu.utils.timer.TaskTimer at 0x7fe83eafac10>,
 '_nan_rate': None}

In [16]:
cpu_inf = automl_gpu.predict(data)

In [17]:
from joblib import dump, load
import time
pickle_file = './gpu.joblib'

In [18]:
start = time.time()
with open(pickle_file, 'wb') as f:
    dump(automl_gpu, f)
raw_dump_duration = time.time() - start
print("Raw dump duration: %0.3fs" % raw_dump_duration)

i am being pickled
Raw dump duration: 0.138s


In [19]:

start = time.time()
with open(pickle_file, 'rb') as f:
    model_inf = load(f)
raw_load_duration = time.time() - start
print("Raw load duration: %0.3fs" % raw_load_duration)


i am being unpickled
dict: {'task': <lightautoml_gpu.tasks.base.Task object at 0x7fe837b650a0>, 'optimization_search_space': {}, 'freeze_defaults': True, 'default_params': {}, '_features': ['freq__n_0006', 'inter__(n_0000__n_0003)', 'inter__(n_0000__n_0004)', 'inter__(n_0000__n_0006)', 'inter__(n_0003__n_0004)', 'inter__(n_0003__n_0006)', 'inter__(n_0004__n_0006)', 'inter__(n_0000__n_0003__n_0004)', 'inter__(n_0000__n_0003__n_0006)', 'inter__(n_0000__n_0004__n_0006)', 'inter__(n_0003__n_0004__n_0006)', 'n_0001', 'n_0002', 'n_0005', 'n_0007', 'ord__n_0000', 'ord__n_0003', 'ord__n_0004'], 'timer': <lightautoml_gpu.utils.timer.TaskTimer object at 0x7fe837b65310>, '_nan_rate': None}
['0', '1', '2']
[<py_boost.utils.tl_wrapper.TLPredictor object at 0x7fe837b65520>, <py_boost.utils.tl_wrapper.TLPredictor object at 0x7fe837b65670>, <py_boost.utils.tl_wrapper.TLPredictor object at 0x7fe837b65790>]
Raw load duration: 0.102s


In [ ]:
model_inf.levels[0][1]

In [ ]:
model_inf.levels[0][1].pre_selection.__dict__

In [20]:
disk_pred = model_inf.predict(data)

In [21]:
disk_pred.data.T

array([[0.0040593 , 0.01104795, 0.00455618, ..., 0.00880432, 0.00519127,
        0.00402712],
       [0.01266458, 0.01693736, 0.01217235, ..., 0.00294561, 0.01033957,
        0.00951653],
       [0.02570421, 0.06104734, 0.02489345, ..., 0.03574519, 0.01928957,
        0.04422379],
       ...,
       [0.59768033, 0.54322815, 0.59243715, ..., 0.3416608 , 0.60686815,
        0.56063   ],
       [0.04642241, 0.05089628, 0.04339997, ..., 0.00870741, 0.04638685,
        0.03446699],
       [0.63145196, 0.58646077, 0.6288966 , ..., 0.36634105, 0.6377027 ,
        0.6012654 ]], dtype=float32)

In [22]:
disk_pred.data.T

array([[0.0040593 , 0.01104795, 0.00455618, ..., 0.00880432, 0.00519127,
        0.00402712],
       [0.01266458, 0.01693736, 0.01217235, ..., 0.00294561, 0.01033957,
        0.00951653],
       [0.02570421, 0.06104734, 0.02489345, ..., 0.03574519, 0.01928957,
        0.04422379],
       ...,
       [0.59768033, 0.54322815, 0.59243715, ..., 0.3416608 , 0.60686815,
        0.56063   ],
       [0.04642241, 0.05089628, 0.04339997, ..., 0.00870741, 0.04638685,
        0.03446699],
       [0.63145196, 0.58646077, 0.6288966 , ..., 0.36634105, 0.6377027 ,
        0.6012654 ]], dtype=float32)

In [23]:
print(cpu_inf.data.T)
print()
print(gpu_inf.data.T)
print()
print(oof_pred_gpu.data.T)
print()
print(oof_pred.data.T)
print()
print(disk_pred.data.T)

[[0.00453568 0.011844   0.00497166 ... 0.00806187 0.00571794 0.00406567]
 [0.01194317 0.01575575 0.01144553 ... 0.00323016 0.0098604  0.00904782]
 [0.02324387 0.05228925 0.02249713 ... 0.0329233  0.01756081 0.03982078]
 ...
 [0.5948076  0.5388622  0.5879661  ... 0.35846284 0.60326093 0.5586468 ]
 [0.04839214 0.05508531 0.04603454 ... 0.00940074 0.04818819 0.03438577]
 [0.63275814 0.58800876 0.6297505  ... 0.3941361  0.6384841  0.60115516]]

[[0.0040593  0.01104795 0.00455619 ... 0.00880433 0.00519127 0.00402713]
 [0.0126646  0.01693738 0.01217235 ... 0.00294561 0.01033958 0.00951654]
 [0.02570421 0.06104737 0.02489345 ... 0.03574518 0.01928957 0.0442238 ]
 ...
 [0.59768033 0.54322815 0.59243715 ... 0.3416608  0.60686815 0.5606301 ]
 [0.04642243 0.05089632 0.0434     ... 0.00870741 0.04638685 0.03446698]
 [0.63145196 0.58646077 0.6288966  ... 0.36634102 0.6377027  0.60126543]]

[[0.00400276 0.01218286 0.00493251 ... 0.00710507 0.00418582 0.00424656]
 [0.01266814 0.0159497  0.01368894 ..

In [ ]:
cluster = LocalCUDACluster(rmm_managed_memory=True, CUDA_VISIBLE_DEVICES="0",
                               protocol="ucx", enable_nvlink=True,
                               memory_limit="8GB")
print("dashboard:", cluster.dashboard_link)
client = Client(cluster)
client.run(cudf.set_allocator, "managed")

In [ ]:
task = Task('multilabel', device='mgpu')

automl_mgpu = TabularAutoMLGPU(
    task = task, 
    timeout = 3600,
    cpu_limit = 1,
    reader_params = {'n_jobs': 1, 'cv': 3, 'random_state': 42, 'npartitions': 2},
    general_params = {'use_algos': [['xgb', 'linear_l2']]},
    client = client
)

oof_pred_mgpu = automl_mgpu.fit_predict(data, roles = roles, verbose = 2)

In [ ]:
automl_mgpu.to_cpu()

In [ ]:
mcpu_inf = automl_mgpu.predict(data)

In [ ]:
print(cpu_inf.data.T)
print()
print(gpu_inf.data.T)
print()
print(oof_pred_gpu.data.T)
print()
print(oof_pred.data.T)
print()
print(oof_pred_mgpu.data.T)
print()
print(mcpu_inf.data.T)

In [ ]:
from joblib import dump, load
import time

pickle_file = './mgpu.joblib'

start = time.time()
with open(pickle_file, 'wb') as f:
    dump(automl_mgpu, f)
raw_dump_duration = time.time() - start
print("Raw dump duration: %0.3fs" % raw_dump_duration)